# SMO implementation
Attempting to implement the SMO (https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tr-98-14.pdf). 

Also consider https://www.csie.ntu.edu.tw/~cjlin/papers/bottou_lin.pdf

First step, we wil need to be able to evaluate the SVM on
a single instance. 

I initially explored hmatrix, but then I realised that I don't need matrix inversion for this algorithm, so using Repa would be better
`--import Numeric.LinearAlgebra`


In [1]:
{- LANGUAGE XTypeOperators -}
import  Data.Array.Repa
import qualified Data.Array.Repa as R


In [2]:
import Data.Array.Repa.Repr.Vector

In [3]:
import Data.Array.Repa.Repr.Vector                   as RV

In [4]:
:i RV.fromListVector

In [5]:
type Value = Double

type BaseVector = Array U DIM1 Value
type BaseScalar = Array U Z Value

--type Sample = BaseVector
--type Weights = BaseVector

-- Making these types to attempt to make the system more type-safe
data Sample = Sample BaseVector
data Weights = Weights BaseVector

weightAsSample :: Weights -> Sample
weightAsSample (Weights w) = Sample w

type Threshold = BaseScalar

-- There is probably an opportunity to build a type-class around the 
-- rules for composing kernel functions. 
type Kernel = Sample -> Sample -> BaseScalar

data ClassLabel = Class1 | Class2 deriving (Show, Eq)

-- An SVM pre
data PredictedLabel = PredictClass1 Value | PredictClass2 Value deriving (Show, Eq)


getLabelValue :: PredictedLabel -> Value
getLabelValue (PredictClass1 v) = v
getLabelValue (PredictClass2 v) = v

predictToTrue :: PredictedLabel -> ClassLabel
predictToTrue (PredictClass1 _) = Class1
predictToTrue (PredictClass2 _) = Class2


chooseClass :: Value -> PredictedLabel
chooseClass res = if res >= 0 then PredictClass1 res else PredictClass2 res
             
dot :: Kernel
dot (Sample a) (Sample b) =  foldS (+) 0 ( a *^ b) 

svm :: Kernel -> Weights -> Threshold -> Sample -> PredictedLabel
svm k w b x = let
                res = ((weightAsSample w) `k` x) -^ b
              in 
                chooseClass  $ res ! Z
                
linearSvm :: Weights -> Threshold -> Sample -> PredictedLabel
linearSvm = svm dot

In [6]:
x_inputs = [1..10] :: [Value]
w_inputs = [11..20] :: [Value]
x = Sample $ fromListUnboxed (Z :. (10::Int))  x_inputs
w = Weights $ fromListUnboxed (Z :. (10::Int))  w_inputs

thresh = fromListUnboxed Z ([2] :: [Value])

svm dot  w thresh  x 

PredictClass1 933.0

In [7]:
largethresh = fromListUnboxed Z ([1000] :: [Value])

svm dot  w largethresh x

PredictClass2 (-65.0)

# Build Helper Functions 
Now that we can evaluate the SVM we need to work towards building the simple functions described in the paper. 

In [8]:
-- equation 13
lowerAlpha :: (Num val, Ord val) => val -> Bool -> val -> val -> val
lowerAlpha c True a1 a2 = max 0 (a1 - a2)
lowerAlpha c False a1 a2 = max 0 (a1 - a2 - c)

-- equation 14
upperAlpha :: (Num val, Ord val) => val -> Bool -> val -> val -> val
upperAlpha c True a1 a2 = min c (c + a1 - a2)
upperAlpha c False a1 a2 = min c (a1 - a2)
                                

In [9]:
-- Make a scalar multiplication operator
(.*) :: Value -> BaseScalar -> BaseScalar
(.*) val vec = R.computeUnboxedS $ R.map (\x -> val*x) vec

-- Make Vector into a scalar
squish :: BaseVector -> BaseScalar
squish v = computeS $ backpermute (Z) (undefined) v 

unsquish :: BaseScalar -> BaseVector
unsquish v = computeS $ backpermute (undefined) (undefined) v

-- equation 15, 2nd Derivative
eta :: Kernel -> Sample -> Sample -> BaseScalar
eta k x1 x2 =  R.computeUnboxedS $ (x1 `k` x1) +^ (x2 `k` x2) -^ (2 .* (x1 `k` x2) )

In [10]:
eta dot x x

AUnboxed Z [0.0]

In [11]:
wrapScalar :: Value -> BaseScalar
wrapScalar s = fromListUnboxed Z ([s] :: [Value])


classToDbl ::  ClassLabel -> Value
classToDbl Class1 = 1
classToDbl Class2 = -1

classToScalar :: ClassLabel -> BaseScalar
classToScalar a = wrapScalar $ classToDbl a


In [12]:
classError :: ClassLabel -> PredictedLabel -> Value
classError trueLabel predLabel =
    let
        predClass = predictToTrue predLabel
        predVal = getLabelValue predLabel
        classVal = classToDbl trueLabel
    in
        if trueLabel == predClass then 0 else classVal - predVal

In [13]:
-- equation 16, minimum along contstraint direction
alpha2New :: Value      -- Current Alpha Value
             -> Value   -- Result of the graidient calculation
             -> ClassLabel   -- TrueLabel of point1
             -> ClassLabel   -- TrueLabel of point2
             -> PredictedLabel   -- Predicted Label of point1
             -> PredictedLabel   -- Predicted Label of point2 
             -> Value   -- New alpha
alpha2New a e y1 y2 s1 s2 = 
    let 
        y2' = classToDbl y2     
        e1 = classError y1 s1   
        e2 = classError y2 s2   
    in
        a + y2' * (e1 - e2) / e -- Implementing eq 16
        

In [14]:
-- Equation 17, 
alphaNewClipped :: (Num a, Ord a) => a -- alpha2New 
                   -> a                -- H
                   -> a                -- L
                   -> a
alphaNewClipped a h l 
    | a >= h = h 
    | a <= l = l
    | otherwise = a

In [15]:
scalarToDbl :: BaseScalar -> Value
scalarToDbl s = s ! ( Z )

calcS :: ClassLabel -> ClassLabel -> Value
calcS y1 y2 =
    classToDbl y1 * classToDbl y2

In [16]:
-- Equation 18

alpha1New :: Value   -- Alpha1
                    -> ClassLabel -- True Label class1
                    -> ClassLabel -- True Label class2
                    -> Value -- Alpha2
                    -> Value -- Alpha2Clipped
                    -> Value
alpha1New a y1 y2 a2 a2clip =
    let 
        s = calcS y1 y2 
    in 
        a + s * (a2 - a2clip)

Equation 19 has a rediculous number of equations!!!


In [17]:
f1 :: Kernel -- Kernel Function
      -> BaseScalar  -- Threshold
      -> ClassLabel  -- True label for class1
      -> ClassLabel  -- True label for class2 
      -> PredictedLabel  -- SVM label for class1
      -> PredictedLabel  -- SVM label for class2  (for consistent interface)
      -> Value      -- Alpha1
      -> Value      -- Alpha2 
      -> Sample      -- sample x1
      -> Sample      -- sample x2
      -> Value
f1 k b y1 y2 t1 _ a1 a2 x1 x2 =
    let
        y1' = classToDbl y1
        y2' = classToDbl y2
        s   = calcS y1 y2
        k11 = scalarToDbl $ x1 `k` x1
        k12 = scalarToDbl $ x1 `k` x2
        b' = scalarToDbl b
        e1 = classError y1 t1
    in
        y1'*(e1 + b') - a1*k11 - s * a2 *k12

In [18]:
f2 :: Kernel -- Kernel Function
      -> BaseScalar  -- Threshold
      -> ClassLabel  -- True label for class1 
      -> ClassLabel  -- True label for class2 
      -> PredictedLabel  -- SVM label for class1 (for consistent interface)
      -> PredictedLabel  -- SVM label for class2
      -> Value      -- Alpha1
      -> Value      -- Alpha2 
      -> Sample      -- sample x1
      -> Sample      -- sample x2
      -> Value
f2 k b y1 y2 _ t2 a1 a2 x1 x2 =
    let
        y1' = classToDbl y1
        y2' = classToDbl y2
        s   = calcS y1 y2
        k12 = scalarToDbl $ x1 `k` x2
        k22 = scalarToDbl $ x2 `k` x2
        b' = scalarToDbl b
        e2 = classError y2 t2
    in
        y2'*(e2 + b') - s * a1 * k12 - a2*k22

In [19]:
l1 :: Value           -- alpha1
      -> Value        -- alpha2
      -> ClassLabel    -- True Label class1
      -> ClassLabel    -- True label class2 
      -> Value        -- lower bound of alpha2
      -> Value
l1 a1 a2 y1 y2 l =
    let
        s = calcS y1 y2
    in
        a1 + s * (a2 - l)
        

In [20]:
h1 :: Value           -- alpha1
     -> Value          -- alpha2
      -> ClassLabel    -- True Label class1
      -> ClassLabel    -- True label class2 
      -> Value        -- upper bound of alpha2
      -> Value
h1 a1 a2 y1 y2 h =
    let
        s = calcS y1 y2
    in
        a1 + s * (a2 - h)
        

In [21]:
psiLower :: Kernel        -- Kernel function
            -> BaseScalar -- Threshold
            -> Value     -- Alpha1
            -> Value     -- Alpha2 
            -> ClassLabel -- True label class1
            -> ClassLabel -- True label class2
            -> Sample     -- x1
            -> Sample     -- x2
            -> PredictedLabel -- SVM predicted sample1
            -> PredictedLabel -- SVM predicted sample2
            -> Value     -- L lower bound
            -> Value
psiLower k b a1 a2 y1 y2 x1 x2 t1 t2 l =
    let
        f1' = f1 k b y1 y2 t1 t2 a1 a2 x1 x2
        f2' = f2 k b y1 y2 t1 t2 a1 a2 x1 x2
        l1' = l1 a1 a2 y1 y2 l
        s = calcS y1 y2
        k11 = scalarToDbl $ x1 `k` x1
        k12 = scalarToDbl $ x1 `k` x2
        k22 = scalarToDbl $ x2 `k` x2
    in
        l1'*f1' + l*f2' 
            + 0.5*l1'*l1'*k11 
            + 0.5*l1'*l1'*k22
            + s*l*l1'*k12
            
            

In [23]:
psiUpper :: Kernel        -- Kernel function
            -> BaseScalar -- Threshold
            -> Value     -- Alpha1
            -> Value     -- Alpha2 
            -> ClassLabel -- True label class1
            -> ClassLabel -- True label class2
            -> Sample     -- x1
            -> Sample     -- x2
            -> PredictedLabel -- SVM predicted sample1
            -> PredictedLabel -- SVM predicted sample2
            -> Value     -- H upper bound
            -> Value
psiUpper k b a1 a2 y1 y2 x1 x2 t1 t2 h =
    let
        f1' = f1 k b y1 y2 t1 t2 a1 a2 x1 x2
        f2' = f2 k b y1 y2 t1 t2 a1 a2 x1 x2
        h1' = l1 a1 a2 y1 y2 h
        s = calcS y1 y2
        k11 = scalarToDbl $ x1 `k` x1
        k12 = scalarToDbl $ x1 `k` x2
        k22 = scalarToDbl $ x2 `k` x2
    in
        h1'*f1' + h*f2' 
            + 0.5*h1'*h1'*k11 
            + 0.5*h1'*h1'*k22
            + s*h*h1'*k12
            

# Understand Heuristic
Ok, so now we need to understand section 2.2, Heuristics for choosing multipliers to optimise.

Hmmm.... by reading section 2.2 I think that I have broken the algorithm somewhat by restricting the SVM output to be assigned a class... it would seem that there is use for the information of how close to the correct class the estimate is... This will need a slight refactor...